In [1]:
import os
import pandas as pd
import json

In [2]:
website_info = pd.read_csv('./../data/Website - Website.csv')
website_info.tail()

,First Name,Last Name,Professional Headshot,Funny Headshot,GitHub,LinkedIn,PDF Resume,Capstone (link),Tagline,Bio,Personal site or Capstone repo,Class Website
22,William,McWilliams,x,NaN,https://github.com/WMcWilliams15,https://www.linkedin.com/in/will-mcwilliams/,NaN,NaN,“It always seems impossible until it is done.”...,"Carpenter turned data analyst, who has an infi...",NaN,NaN
23,Yasin,Hasan,x,NaN,https://github.com/yhasan47,https://www.linkedin.com/in/yasinhasan47,NaN,NaN,"""I am ready to face any challenge that might b...",A Science Teacher turned into a Data Analyst.,NaN,NaN
24,UrLeaka,Test,x,x,https://github.com/unewsome,https://www.linkedin.com/in/urleaka-newsome/,x,NaN,"""Data is as data does""",This stuff is fun!' I can't believe I said tha...,NaN,https://nss-data-science-cohort-2.github.io/
25,Matt,Test,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Taylor,Test,NaN,NaN,NaN,NaN,NaN,NaN,"""Data does as data is""",NaN,NaN,NaN


---

Filter out the teachers / nan stuff

In [3]:
website_info = website_info[~((website_info['Last Name'] == 'Test') | website_info['First Name'].isnull())]

---

Rename the columns to something that can be ingested by the js

In [4]:
column_map = {
    'First Name': 'firstName',
    'Last Name': 'lastName',
    'Tagline': 'reelThemIn',
    'Bio': 'bio',
    'GitHub': 'github',
    'LinkedIn': 'linkedin',
    'Personal site or Capstone repo': 'portfolio'
}

website_info = website_info.rename(column_map, axis=1)
website_info = website_info[column_map.values()]

website_info.columns

Index(['firstName', 'lastName', 'reelThemIn', 'bio', 'github', 'linkedin',
       'portfolio'],
      dtype='object')

---

Make the paragraphs

In [5]:
def html_paragraph(v):
    if pd.isna(v):
        return ""
    return f"<p>{str(v)}</p>"


website_info['reelThemIn'] = website_info['reelThemIn'].apply(html_paragraph)
website_info['bio'] = website_info['bio'].apply(html_paragraph)

---

Clean up some excess characters

In [6]:
for col in website_info.columns:
    website_info[col] = website_info[col] \
        .str.replace('“', '"') \
        .str.replace('”', '"') \
        .str.replace("’", "'") \
        .str.replace("—", "-")

---

Gather the image paths

In [7]:
def image_path(name, idx):
    return f"../assets/img/resized/{name.lower()}{idx}.jpeg"


website_info['proImg'] = website_info['firstName'].apply(image_path, idx=1)
website_info['funImg'] = website_info['firstName'].apply(image_path, idx=2)

---

Assign Ids

In [8]:
website_info['id'] = website_info.index + 1

---

Take a peek

In [9]:
website_info.head(1)

,firstName,lastName,reelThemIn,bio,github,linkedin,portfolio,proImg,funImg,id
0,Adam,Tsuchiyama,"<p>""In God we trust. All others, bring data."" ...",<p>Whether it's determining how to incorporate...,https://github.com/adamtsuch21,https://www.linkedin.com/in/adam-tsuchiyama/,NaN,../assets/img/resized/adam1.jpeg,../assets/img/resized/adam2.jpeg,1


In [10]:
website_info.to_json("./../data/cohort.json", orient='records')